# Import librerias

In [1]:
import requests
import json
import pandas as pd
from collections import Counter
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from datetime import datetime
import http.client
from pandas import json_normalize


In [2]:
print("Trabajaremos con las siguientes versiones\n")
print("Pandas",pd.__version__)
print("Numpy",np.__version__)
print("Seaborn", sns.__version__)
!python --version

Trabajaremos con las siguientes versiones

Pandas 2.0.2
Numpy 1.25.0
Seaborn 0.12.2
Python 3.11.3


# Funciones

In [3]:
def import_data(resource_id):

    #Define base url and parameters
    url = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search'
    params = {'resource_id': resource_id, 'limit': 100000000000000} # Change limit size of the retrieved rows: https://docs.ckan.org/en/latest/maintaining/datastore.html#the-data-api  (see parameters in the datastore_search request)

    #Conect to API
    response_API = requests.get(url, params=params)
    if response_API.status_code != 200:
        raise Exception('Could not get data from the API')

    #Get data from API
    data = response_API.json()

    #Get data records
    data_records = data['result']['records']

    #Convert json into a dataframe
    df = pd.json_normalize(data_records)

    return df

In [4]:
# Creamos una función simple para hacer un report simple sobre los DataFrame suministrados
def report_df(dataframe, verbose = True):
    print("El tamaño del dataset es", dataframe.shape[0], "filas y", dataframe.shape[1], "columnas")
    print()
    print(dataframe.info(verbose = verbose))
    total_nulos = dataframe.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")
    print()
    print("Hay", dataframe.duplicated().sum(), "duplicados")


# Load Data Cens

In [5]:
df_cens = import_data('c933fa10-a965-4509-8de6-e519281dc245').drop(columns = '_id')
df_cens.head()


,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb
2,4584520,La Llave de oro,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,inmo,4584520,3644426DF3834D,10,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,91,433678,31/03/2014,2012-05-16,Sant Martí,"433678,8",serv
3,"4584515,5",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584515,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,91,433664,31/03/2014,2012-05-16,Sant Martí,"433664,4",locb
4,"4584511,1",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584511,3644426DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,93,433649,31/03/2014,2012-05-16,Sant Martí,"433649,9",locb


# EDA

In [6]:
## Observamos los datos que tenemos 
report_df(df_cens)

df_cens.head(2)

El tamaño del dataset es 67433 filas y 19 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67433 entries, 0 to 67432
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Y_ED50         67433 non-null  object
 1   NOM            67433 non-null  object
 2   CBARRI         67433 non-null  object
 3   NBARRI         67433 non-null  object
 4   CARRCADAST     67433 non-null  object
 5   CODI_EIXOS     67433 non-null  object
 6   Y              67433 non-null  object
 7   RCNUMBER       67433 non-null  object
 8   CDISTRI        67433 non-null  object
 9   BCN3           67433 non-null  object
 10  BCN2           67433 non-null  object
 11  BCN1           67433 non-null  object
 12  DOORNUM        67433 non-null  object
 13  X              67433 non-null  object
 14  ULTIMA_VISITA  67433 non-null  object
 15  DATAINICI      67433 non-null  object
 16  NDISTRIC       67433 non-null  object
 17  X_ED50         67

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
0,"4584527,7",Orange,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,movi,4584527,3644423DF3834D,10,ALTRES,ALTRES,SERVEIS,89,433707,31/03/2014,2012-05-16,Sant Martí,"433707,7",serv
1,"4584524,4",Local buit,69,Diagonal Mar i el Front Marítim del Poblenou,AV DIAGONAL,localb,4584524,3644423DF3834D,10,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,89,433701,31/03/2014,2012-05-16,Sant Martí,"433701,1",locb


In [7]:
# Examinamos los duplicados existentes
df_cens[df_cens.duplicated()]

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
125,"4583986,6",Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583986,1740810DF3814B,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,298,431770,31/03/2014,2013-12-09,Eixample,431770,locb
150,"4583601,1",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583601,0835306DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,319,430826,31/03/2014,2013-12-31,Eixample,"430826,6",locb
179,4583750,Local Buit,06,la Sagrada Família,AV DIAGONAL,localb,4583750.37,1037609DF3813G,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,336,431055.96,31/03/2014,2011-07-06,Eixample,"431055,5",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
266,"4583499,9",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,4583499.89,0235724DF3803E,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,414,430274.69,31/03/2014,2011-07-06,Eixample,"430274,4",locb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67374,"4588284,4",Varietats,48,la Guineueta,RB CAÃADOR,variet,4588284,0682301DF3808D,08,ALTRES,ALTRES,ALTRES,1,430536,09/03/2014,2014-03-09,Nou Barris,"430536,6",ALTRES
67387,"4582415,511",LLI privada Ninets,08,l Antiga Esquerra de l Eixample,ValÞncia,educ,45824151104439,,02,ENSENYAMENT,SERVEIS,SERVEIS,218-220,42971322259258,16/04/2013,2013-04-16,Eixample,"429713,322",other
67393,"4587906,6",Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587906,0479903DF3807G,08,ALTRES,ALTRES,ALTRES,474,430454,03/03/2014,2014-03-09,Nou Barris,"430454,4",ALTRES
67400,4587970,Varietats,48,la Guineueta,RB FABRA I PUIG,variet,4587970,0479902DF3807G,08,ALTRES,ALTRES,ALTRES,478,430463,03/03/2014,2014-03-09,Nou Barris,"430463,3",ALTRES


In [8]:
# Comprobamos si estos duplicados, realmente lo son
df_cens[df_cens['RCNUMBER'] == '0837728DF3803F']

,Y_ED50,NOM,CBARRI,NBARRI,CARRCADAST,CODI_EIXOS,Y,RCNUMBER,CDISTRI,BCN3,BCN2,BCN1,DOORNUM,X,ULTIMA_VISITA,DATAINICI,NDISTRIC,X_ED50,M_CODI_EIXOS
207,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb
208,"4583706,654",Local Buit,07,la Dreta de l Eixample,AV DIAGONAL,localb,45837065477,0837728DF3803F,02,LOCAL BUIT,LOCAL BUIT,LOCAL BUIT,358,43086864201411,31/03/2014,2013-12-31,Eixample,"430868,864",locb


In [9]:
# Decidimos eliminar duplicados, viendo que tiene sentido
df_cens.drop_duplicates(inplace=True)

In [10]:
# Eliminamos columnas que no hacen falta
df_cens =  df_cens.drop(['Y_ED50', 'X_ED50', 'X', 'Y', 
                         'CARRCADAST', 'DOORNUM',
                         'ULTIMA_VISITA', 'DATAINICI',
                         'RCNUMBER'], axis=1)

In [11]:
# Comprobamos los datos que contenemos 
df_cens.describe(exclude=np.number).T

,count,unique,top,freq
NOM,66444,45556,Local Buit,9228
CBARRI,66444,74,07,4142
NBARRI,66444,74,la Dreta de l Eixample,4142
CODI_EIXOS,66444,120,localb,9715
CDISTRI,66444,11,02,14931
BCN3,66444,41,ALTRES,17930
BCN2,66444,9,SERVEIS,30785
BCN1,66444,4,SERVEIS,31244
NDISTRIC,66444,11,Eixample,14931
M_CODI_EIXOS,66444,11,serv,16895


In [12]:
# Probamos los valores que contiene esta columna para entender mejor nuestro dataset
df_cens['NBARRI'].value_counts()

NBARRI
la Dreta de l Eixample             4142
la Vila de Gràcia                  3481
Sant Gervasi - Galvany             2947
l Antiga Esquerra de l Eixample    2810
el Raval                           2540
                                   ... 
#N/A                                 21
Baró de Viver                        21
Torre Baró                           20
Vallbona                              8
la Clota                              6
Name: count, Length: 74, dtype: int64

In [13]:
# Remplazamos los valores nulos para que se puedan detectar
df_cens.replace('#N/A',np.nan,inplace=True)

In [14]:
# Comprobamos que se hizo el cambio correctamente
df_cens[df_cens.isnull().any(axis=1)].head(2)

,NOM,CBARRI,NBARRI,CODI_EIXOS,CDISTRI,BCN3,BCN2,BCN1,NDISTRIC,M_CODI_EIXOS
3556,CLUB DE PETANCA CASA OLIVA,NaN,NaN,espo,NaN,ALTRES,ALTRES,ALTRES,NaN,ALTRES
14999,Institut Lluís Vives,NaN,NaN,educ,NaN,ENSENYAMENT,SERVEIS,SERVEIS,NaN,other


In [15]:
# Eliminamos valores nulos
df_cens.dropna(inplace=True)
df_cens.reset_index(drop=True,inplace=True)

In [16]:
# Renombramos columnas para trabajar más fácilmente
df_cens.rename(columns={'CBARRI': 'Codi_Barri', 'NBARRI': 'Nom_Barri', 'CDISTRI': 'Codi_Districte', 'NDISTRIC': 'Nom_Districte'}, inplace=True)

In [17]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NOM             66423 non-null  object
 1   Codi_Barri      66423 non-null  object
 2   Nom_Barri       66423 non-null  object
 3   CODI_EIXOS      66423 non-null  object
 4   Codi_Districte  66423 non-null  object
 5   BCN3            66423 non-null  object
 6   BCN2            66423 non-null  object
 7   BCN1            66423 non-null  object
 8   Nom_Districte   66423 non-null  object
 9   M_CODI_EIXOS    66423 non-null  object
dtypes: object(10)
memory usage: 5.1+ MB


In [18]:
df_cens[df_cens['Nom_Barri'].str.contains('la Nova')]

,NOM,Codi_Barri,Nom_Barri,CODI_EIXOS,Codi_Districte,BCN3,BCN2,BCN1,Nom_Districte,M_CODI_EIXOS
423,Departament De Treball I Sostenibilitat Genera...,09,la Nova Esquerra de l Eixample,admi,02,ALTRES,ALTRES,ALTRES,Eixample,ALTRES
436,Parc De Neteja Ajuntament De Barcelona,09,la Nova Esquerra de l Eixample,admi,02,ALTRES,ALTRES,ALTRES,Eixample,ALTRES
488,LLI privada Babet,09,la Nova Esquerra de l Eixample,educ,02,ENSENYAMENT,SERVEIS,SERVEIS,Eixample,other
489,Arco,09,la Nova Esquerra de l Eixample,educ,02,ENSENYAMENT,SERVEIS,SERVEIS,Eixample,other
496,LLI privada I-Beth,09,la Nova Esquerra de l Eixample,educ,02,ENSENYAMENT,SERVEIS,SERVEIS,Eixample,other
...,...,...,...,...,...,...,...,...,...,...
65973,LLI privada Pam i Pipa,09,la Nova Esquerra de l Eixample,educ,02,ENSENYAMENT,SERVEIS,SERVEIS,Eixample,other
66118,Ambit Surban,09,la Nova Esquerra de l Eixample,variet,02,ALTRES,ALTRES,ALTRES,Eixample,ALTRES
66138,Pinocho,09,la Nova Esquerra de l Eixample,educ,02,ENSENYAMENT,SERVEIS,SERVEIS,Eixample,other
66176,Ocal,09,la Nova Esquerra de l Eixample,variet,02,ALTRES,ALTRES,ALTRES,Eixample,ALTRES


In [19]:
# Comprobamos diferencias entre columnas BCN3, BCN2 y BCN1
df_cens[(df_cens['NOM']=='Local Buit') & (df_cens['BCN2']!= 'LOCAL BUIT')]

,NOM,Codi_Barri,Nom_Barri,CODI_EIXOS,Codi_Districte,BCN3,BCN2,BCN1,Nom_Districte,M_CODI_EIXOS
2912,Local Buit,27,el Putxet i el Farró,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
3833,Local Buit,19,les Corts,ofic,04,ALTRES,SERVEIS,SERVEIS,Les Corts,serv
9869,Local Buit,25,Sant Gervasi - la Bonanova,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
11022,Local Buit,26,Sant Gervasi - Galvany,moda,05,VESTIR,EQUIPAMENT PERSONAL,COMERÇ AL DETALL,Sarrià-Sant Gervasi,pers
12407,Local Buit,17,Sants - Badal,colma,03,ALIMENTACIÓ,QUOTIDIÀ ALIMENTARI,COMERÇ AL DETALL,Sants-Montjuïc,quot
12413,Local Buit,46,el Turó de la Peira,locu,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
15234,Local Buit,60,Sant Andreu,tall,09,REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS),SERVEIS,SERVEIS,Sant Andreu,tran
16017,Local Buit,18,Sants,inmo,03,ACTIVITATS IMMOBILIÀRIES,SERVEIS,SERVEIS,Sants-Montjuïc,serv
18470,Local Buit,44,Vilapicina i la Torre Llobeta,ofic,08,ALTRES,SERVEIS,SERVEIS,Nou Barris,serv
19589,Local Buit,18,Sants,arranj,03,"MANTENIMENT, NETEJA I PRODUCCIÓ",SERVEIS,SERVEIS,Sants-Montjuïc,serv


In [20]:
# Entender mejor los datos que contiene, vemos que se parece a las columnas de BCN3,2 y 1
df_cens['M_CODI_EIXOS'].value_counts()

M_CODI_EIXOS
serv      16895
host      10249
locb       9715
pers       6835
quot       6104
ALTRES     4069
altr       2958
llar       2785
tran       2522
cult       2499
other      1792
Name: count, dtype: int64

In [21]:
df_cens['BCN1'].value_counts()

BCN1
SERVEIS             31239
COMERÇ AL DETALL    21400
LOCAL BUIT           9715
ALTRES               4069
Name: count, dtype: int64

In [22]:
df_cens['BCN3'].value_counts()

BCN3
ALTRES                                                            17914
RESTAURANTS, BARS I HOTELS (INCLÒS HOSTALS, PENSIONS I FONDES)    10249
LOCAL BUIT                                                         9715
VESTIR                                                             3488
ALIMENTACIÓ                                                        2406
ENSENYAMENT                                                        2239
PA, PASTISSERIA I LÀCTICS                                          1705
MOBLES I ARTICLES FUSTA I METALL                                   1639
FINANCES I ASSEGURANCES                                            1551
SANITAT I ASSISTÈNCIA                                              1385
REPARACIONS (ELECTRODOMÈSTICS I AUTOMÒBILS)                        1283
LLIBRES, DIARIS I REVISTES                                         1125
FARMÀCIES                                                          1065
FRUITES I VERDURES                                         

In [23]:
df_cens['BCN2'].value_counts()

BCN2
SERVEIS                   30780
LOCAL BUIT                 9715
QUOTIDIÀ ALIMENTARI        6569
ALTRES                     6277
EQUIPAMENT PERSONAL        5114
PARAMENT DE LA LLAR        3086
OCI I CULTURA              2798
QUOTIDIÀ NO ALIMENTARI     1724
AUTOMOCIÓ                   360
Name: count, dtype: int64

In [24]:
df_cens.columns

Index(['NOM', 'Codi_Barri', 'Nom_Barri', 'CODI_EIXOS', 'Codi_Districte',
       'BCN3', 'BCN2', 'BCN1', 'Nom_Districte', 'M_CODI_EIXOS'],
      dtype='object')

In [25]:
# Eliminamos más columnas que vimos que contiene valores parecidos para reducir nuestro dataset de trabajo
df_cens.drop(['NOM','CODI_EIXOS','BCN3','BCN1','M_CODI_EIXOS'],axis=1,inplace=True)

In [26]:
df_cens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Barri      66423 non-null  object
 1   Nom_Barri       66423 non-null  object
 2   Codi_Districte  66423 non-null  object
 3   BCN2            66423 non-null  object
 4   Nom_Districte   66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [27]:
# Cambio nombre columna por un nombre más adecuado al contenido
df_cens.rename(columns = {'BCN2':'Activitat_Economica'},inplace=True)

In [28]:
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí


In [29]:
report_df(df_cens)

El tamaño del dataset es 66423 filas y 5 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66423 entries, 0 to 66422
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Codi_Barri           66423 non-null  object
 1   Nom_Barri            66423 non-null  object
 2   Codi_Districte       66423 non-null  object
 3   Activitat_Economica  66423 non-null  object
 4   Nom_Districte        66423 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB
None

Tenemos un total de 0 nulos

Hay 65818 duplicados


In [30]:
# Groupby para calcular el total de actividades que hay según el barrio y el tipo de actividad economica
Activitat_Barris = df_cens.groupby(['Codi_Barri','Activitat_Economica']).size().to_frame(name = 'Total_Activitat_Per_Barri').reset_index()

In [31]:
# Imprimimos el dataframe generado para ver como queda y ver si tiene  sentido
Activitat_Barris

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
0,01,ALTRES,381
1,01,AUTOMOCIÓ,1
2,01,EQUIPAMENT PERSONAL,225
3,01,LOCAL BUIT,245
4,01,OCI I CULTURA,88
...,...,...,...
600,73,OCI I CULTURA,26
601,73,PARAMENT DE LA LLAR,43
602,73,QUOTIDIÀ ALIMENTARI,116
603,73,QUOTIDIÀ NO ALIMENTARI,28


In [32]:
# Comprobación visual de un barrio aleatorio
Activitat_Barris[Activitat_Barris['Codi_Barri']=='69']

,Codi_Barri,Activitat_Economica,Total_Activitat_Per_Barri
561,69,ALTRES,20
562,69,EQUIPAMENT PERSONAL,8
563,69,LOCAL BUIT,42
564,69,OCI I CULTURA,17
565,69,PARAMENT DE LA LLAR,3
566,69,QUOTIDIÀ ALIMENTARI,19
567,69,QUOTIDIÀ NO ALIMENTARI,11
568,69,SERVEIS,185


In [33]:
# merge del dataframe original con el dataframe generado con el groupby anterior. Comprobamos que no se haga un cartesiano
print(df_cens.shape)

df_cens = pd.merge(df_cens,Activitat_Barris, on = ['Codi_Barri','Activitat_Economica'], how = 'left')

print(df_cens.shape)

(66423, 5)
(66423, 6)


In [34]:
# imprimimos por pantalla para comprobar como queda
df_cens.head()

,Codi_Barri,Nom_Barri,Codi_Districte,Activitat_Economica,Nom_Districte,Total_Activitat_Per_Barri
0,69,Diagonal Mar i el Front Marítim del Poblenou,10,ALTRES,Sant Martí,20
1,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
2,69,Diagonal Mar i el Front Marítim del Poblenou,10,SERVEIS,Sant Martí,185
3,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42
4,69,Diagonal Mar i el Front Marítim del Poblenou,10,LOCAL BUIT,Sant Martí,42


In [35]:
# pivot table para que las actividades economicas se coniertan en columnas y reducir un poco la dimensionalidad
df_cens_pivot = pd.pivot_table(df_cens, values='Total_Activitat_Per_Barri', index=['Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri'], columns='Activitat_Economica', fill_value=0)

In [36]:
df_cens_pivot.reset_index(inplace=True)

In [37]:
# cambio de nombre de columnas para estar alineados con el equipo de Back
df_cens_pivot.rename(columns = {
    'Codi_Districte':'codiDistricte',
    'Nom_Districte' : 'nomDistricte',
    'Nom_Barri':'nomBarri',
    'Codi_Barri':'codiBarri',
    'Nom_Barri':'nomBarri',
    'ALTRES':'nAltres',
    'AUTOMOCIÓ':'nAutomocio',
    'EQUIPAMENT PERSONAL':'nEquipamentPersonal',
    'LOCAL BUIT' : 'nLocalBuit',
    'OCI I CULTURA': 'nOciCultura',
    'PARAMENT DE LA LLAR':'nParamentLlar',
    'QUOTIDIÀ ALIMENTARI' : 'nQuotidiaAlimentari',
    'QUOTIDIÀ NO ALIMENTARI' : 'nQuotidiaNoAlimentari',
    'SERVEIS':'nServeis'
    
},inplace=True

)

In [38]:
# generamos una nueva columna con el año del dataset para más tarde añadir otro años
df_cens_pivot['Anio'] = '2014'

In [39]:
df_cens_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   codiDistricte          73 non-null     object
 1   nomDistricte           73 non-null     object
 2   codiBarri              73 non-null     object
 3   nomBarri               73 non-null     object
 4   nAltres                73 non-null     int64 
 5   nAutomocio             73 non-null     int64 
 6   nEquipamentPersonal    73 non-null     int64 
 7   nLocalBuit             73 non-null     int64 
 8   nOciCultura            73 non-null     int64 
 9   nParamentLlar          73 non-null     int64 
 10  nQuotidiaAlimentari    73 non-null     int64 
 11  nQuotidiaNoAlimentari  73 non-null     int64 
 12  nServeis               73 non-null     int64 
 13  Anio                   73 non-null     object
dtypes: int64(9), object(5)
memory usage: 8.1+ KB


In [40]:
# Convertimos el año en formato fecha
df_cens_pivot['Anio'] = pd.to_datetime(df_cens_pivot['Anio'],format = "%Y")

In [41]:
df_cens_pivot

Activitat_Economica,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nLocalBuit,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio
0,01,Ciutat Vella,01,el Raval,381,1,225,245,88,137,296,47,1120,2014-01-01
1,01,Ciutat Vella,02,el Barri Gòtic,315,0,478,199,78,76,161,37,786,2014-01-01
2,01,Ciutat Vella,03,la Barceloneta,99,0,26,108,24,13,89,16,370,2014-01-01
3,01,Ciutat Vella,04,"Sant Pere, Santa Caterina i la Ribera",259,1,363,155,53,98,175,34,688,2014-01-01
4,02,Eixample,05,el Fort Pienc,81,12,40,199,47,55,113,33,556,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,20,0,8,42,17,3,19,11,185,2014-01-01
69,10,Sant Martí,70,el Besòs i el Maresme,90,1,16,59,12,22,71,18,204,2014-01-01
70,10,Sant Martí,71,Provençals del Poblenou,166,5,11,122,26,16,40,19,314,2014-01-01
71,10,Sant Martí,72,Sant Martí de Provençals,80,4,40,120,23,34,84,18,336,2014-01-01


# Load Data Prices M2

In [42]:
connect_opendata = http.client.HTTPSConnection("opendata-ajuntament.barcelona.cat")

headers = {
    'cache-control': "no-cache"
    }

connect_opendata.request("GET", "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=e42cf2cf-a76e-4a32-9357-cf90e0ea8ead", headers=headers)

response = connect_opendata.getresponse()
data = response.read()

json_data = json.loads(data.decode('utf-8'))

# Check if the JSON data is nested and needs flattening
if 'result' in json_data and 'records' in json_data['result']:
    # Flatten the JSON data and create a DataFrame
    precio_venta_m2 = json_normalize(json_data['result']['records'])
else:
    # Convert the relevant part of your JSON data to a JSON string and read into a DataFrame
    precio_venta_m2 = pd.read_json(json.dumps(json_data['result']['records']))



# Using regular expression to remove the pattern
precio_venta_m2['BARRIS'] = precio_venta_m2['BARRIS'].str.replace(r'^\d+\.\s+', '', regex=True)



precio_venta_m2.columns = ['Nom_Barri', '2011', '2008', '2010','DTE','2009','ID']

precio_venta_m2 = precio_venta_m2.drop(columns=['DTE', 'ID'])

new_order = ['Nom_Barri', '2008', '2009', '2010','2011']
precio_venta_m2 = precio_venta_m2[new_order]

precio_venta_m2.to_csv("precio_venta_m2.csv")

# Display the DataFrame
precio_venta_m2.head()


,Nom_Barri,2008,2009,2010,2011
0,el Raval,3.065,2.773,2.445,2.176
1,el Barri Gòtic,3.828,3.750,3.026,2.632
2,la Barceloneta,--,--,3.005,2.135
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482
4,el Fort Pienc,2.893,2.736,2.422,2.224


In [43]:
# Convert year columns to numeric
year_columns = precio_venta_m2.columns[1:]  # Exclude the first two and last two columns
precio_venta_m2[year_columns] = precio_venta_m2[year_columns].apply(pd.to_numeric, errors='coerce')

# Calculate the average price per sqm over the years for each neighborhood
precio_venta_m2['Average_Price'] = precio_venta_m2[year_columns].mean(axis=1)

precio_venta_m2.to_csv("precio_venta_m2.csv")

# Display the updated dataframe
precio_venta_m2.head()


,Nom_Barri,2008,2009,2010,2011,Average_Price
0,el Raval,3.065,2.773,2.445,2.176,2.61475
1,el Barri Gòtic,3.828,3.750,3.026,2.632,3.30900
2,la Barceloneta,NaN,NaN,3.005,2.135,2.57000
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482,2.97650
4,el Fort Pienc,2.893,2.736,2.422,2.224,2.56875


In [44]:
# Descriptive statistics for the dataset
descriptive_stats = precio_venta_m2.describe()

# Display descriptive statistics
descriptive_stats

,2008,2009,2010,2011,Average_Price
count,56.000000,53.000000,56.00000,58.000000,61.000000
mean,2.705982,2.440208,2.28775,2.004810,2.330217
std,0.685203,0.546705,0.55166,0.464295,0.538360
min,1.491000,1.387000,1.28900,1.110000,1.432250
25%,2.174250,2.058000,1.90950,1.681750,1.900250
50%,2.670500,2.392000,2.28250,1.974000,2.251500
75%,3.142750,2.773000,2.66175,2.264000,2.672750
max,4.326000,3.750000,3.98900,3.351000,3.848250


The year 2009 has the most missing values (20 instances).
The year 2011 has the fewest missing values (15 instances).
Missing data in other years ranges from 15 to 20 instances.
Now, I'll provide a brief overview of this dataset, including descriptive statistics and any other interesting observations. Let's start with the descriptive statistics. ​​

The descriptive statistics provide insight into the price data for commercial properties in Barcelona from 2008 to 2011. Here are some key observations:

Count: The dataset contains 73 neighborhoods. However, not all years have data for every neighborhood, as indicated by the varying counts (56 for 2008, 53 for 2009, 56 for 2010, and 58 for 2011).

Mean Prices:

The average price per square meter was highest in 2008 (≈2.706 ≈2.706 EUR), and it decreased each subsequent year, reaching its lowest in 2011 (≈2.005≈2.005 EUR).

The overall average price across all years and neighborhoods is approximately 2.330 EUR.
Standard Deviation:

The standard deviation indicates variability in prices across neighborhoods. The highest variability was in 2008 (≈0.685≈0.685) and the lowest in 2011 (≈0.464≈0.464).

Minimum and Maximum Prices:

The minimum price per square meter fluctuated from 1.491 EUR in 2008 to 1.110 EUR in 2011.
The maximum price showed a decreasing trend, from 4.326 EUR in 2008 to 3.351 EUR in 2011.
Quartiles:

The 25th, 50th (median), and 75th percentiles also exhibit a decreasing trend across the years.
This analysis reveals a general decrease in the price per square meter for commercial properties in Barcelona's neighborhoods from 2008 to 2011. The variability in prices also decreased over this period, suggesting a possible stabilization or uniformity in property values across different areas.




In [45]:
precio_venta_m2.drop(['Average_Price'], axis=1)


,Nom_Barri,2008,2009,2010,2011
0,el Raval,3.065,2.773,2.445,2.176
1,el Barri Gòtic,3.828,3.750,3.026,2.632
2,la Barceloneta,NaN,NaN,3.005,2.135
3,Sant Pere Santa Caterina i la Ribera,3.450,3.146,2.828,2.482
4,el Fort Pienc,2.893,2.736,2.422,2.224
...,...,...,...,...,...
68,Diagonal Mar i el Front Marítim del Poblenou,3.752,2.445,2.991,2.651
69,el Besòs i el Maresme,3.669,2.842,2.751,2.121
70,Provençals del Poblenou,3.857,2.660,2.516,2.255
71,Sant Martí de Provençals,1.957,2.266,2.175,1.812


In [46]:
precio_venta_m2=pd.melt(precio_venta_m2, id_vars=['Nom_Barri'], value_vars=['2008', '2009','2010','2011'])

precio_venta_m2.columns = ['Nom_Barri', 'Anio', 'Price_m2']
precio_venta_m2['Nom_Barri'] = precio_venta_m2['Nom_Barri'].str.lower()

precio_venta_m2

,Nom_Barri,Anio,Price_m2
0,el raval,2008,3.065
1,el barri gòtic,2008,3.828
2,la barceloneta,2008,NaN
3,sant pere santa caterina i la ribera,2008,3.450
4,el fort pienc,2008,2.893
...,...,...,...
287,diagonal mar i el front marítim del poblenou,2011,2.651
288,el besòs i el maresme,2011,2.121
289,provençals del poblenou,2011,2.255
290,sant martí de provençals,2011,1.812


# Load Data Rent Prices

In [47]:
connect_opendata = http.client.HTTPSConnection("opendata-ajuntament.barcelona.cat")

headers = {
    'cache-control': "no-cache"
    }

connect_opendata.request("GET", "https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search?resource_id=97356d26-30b9-436a-8dbb-d0d05f0a87fd", headers=headers)

response = connect_opendata.getresponse()
data = response.read()

json_data = json.loads(data.decode('utf-8'))

# Check if the JSON data is nested and needs flattening
if 'result' in json_data and 'records' in json_data['result']:
    # Flatten the JSON data and create a DataFrame
    precio_alquiler = json_normalize(json_data['result']['records'])
else:
    # Convert the relevant part of your JSON data to a JSON string and read into a DataFrame
    precio_alquiler = pd.read_json(json.dumps(json_data['result']['records']))



# Using regular expression to remove the pattern
precio_alquiler['BARRIS'] = precio_alquiler['BARRIS'].str.replace(r'^\d+\.\s+', '', regex=True)



precio_alquiler.columns = ['Nom_Barri', '2011', '2008', '2010','DTE','2009','ID']

precio_alquiler = precio_alquiler.drop(columns=['DTE', 'ID'])

new_order = ['Nom_Barri', '2008', '2009', '2010','2011']
precio_alquiler = precio_alquiler[new_order]

precio_alquiler.to_csv("precio_alquiler.csv")

# Display the DataFrame
precio_alquiler.head()

,Nom_Barri,2008,2009,2010,2011
0,el Raval,1543,1362,1216,1189
1,el Barri Gòtic,1859,1481,1369,1325
2,la Barceloneta,1469,1526,1660,1648
3,Sant Pere Santa Caterina i la Ribera,1585,1374,1291,1225
4,el Fort Pienc,1243,1098,1000,921


In [48]:
precio_alquiler=pd.melt(precio_alquiler, id_vars=['Nom_Barri'], value_vars=['2008', '2009','2010','2011'])

precio_alquiler.columns = ['Nom_Barri', 'Anio', 'Price']
precio_alquiler['Nom_Barri'] = precio_alquiler['Nom_Barri'].str.lower()

precio_alquiler

,Nom_Barri,Anio,Price
0,el raval,2008,1543
1,el barri gòtic,2008,1859
2,la barceloneta,2008,1469
3,sant pere santa caterina i la ribera,2008,1585
4,el fort pienc,2008,1243
...,...,...,...
287,diagonal mar i el front marítim del poblenou,2011,1126
288,el besòs i el maresme,2011,--
289,provençals del poblenou,2011,832
290,sant martí de provençals,2011,976


In [49]:
df_precios = pd.merge(precio_alquiler, precio_venta_m2, on=['Nom_Barri', 'Anio'])
df_precios

,Nom_Barri,Anio,Price,Price_m2
0,el raval,2008,1543,3.065
1,el barri gòtic,2008,1859,3.828
2,la barceloneta,2008,1469,NaN
3,sant pere santa caterina i la ribera,2008,1585,3.450
4,el fort pienc,2008,1243,2.893
...,...,...,...,...
287,diagonal mar i el front marítim del poblenou,2011,1126,2.651
288,el besòs i el maresme,2011,--,2.121
289,provençals del poblenou,2011,832,2.255
290,sant martí de provençals,2011,976,1.812


In [50]:
df_precios_2011 = df_precios[df_precios['Anio'] == '2011']

In [51]:
df_precios_2011.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 219 to 291
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Nom_Barri  73 non-null     object 
 1   Anio       73 non-null     object 
 2   Price      73 non-null     object 
 3   Price_m2   58 non-null     float64
dtypes: float64(1), object(3)
memory usage: 2.9+ KB


In [52]:
# Remplazamos los valores de año para igualar al 2014 y poder merge con el otro df
df_precios_2011.replace('2011','2014',inplace=True)

C:\Users\chuec\AppData\Local\Temp\ipykernel_7432\155332974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precios_2011.replace('2011','2014',inplace=True)


In [53]:
# Comprobamos que los cambios se han realizado correctamente
df_precios_2011['Anio'].value_counts()

Anio
2014    73
Name: count, dtype: int64

In [54]:
# cambiamos el nombre para que haga referencia al cambio anterior
df_precios_2014 = df_precios_2011

In [55]:
df_precios_2014['Anio'] = pd.to_datetime(df_precios_2014['Anio'],format = '%Y')

C:\Users\chuec\AppData\Local\Temp\ipykernel_7432\3453662360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precios_2014['Anio'] = pd.to_datetime(df_precios_2014['Anio'],format = '%Y')


In [56]:
df_precios_2014.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 219 to 291
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Nom_Barri  73 non-null     object        
 1   Anio       73 non-null     datetime64[ns]
 2   Price      73 non-null     object        
 3   Price_m2   58 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.9+ KB


In [57]:
df_precios_2014.head(2)

,Nom_Barri,Anio,Price,Price_m2
219,el raval,2014-01-01,1189,2.176
220,el barri gòtic,2014-01-01,1325,2.632


In [58]:
# cambio de nombre de columnas para estar alineados con el equipo de Back
df_precios_2014.rename(columns = {
    'Nom_Barri':'nomBarri',
    'Price':'preuMitja',
    'Price_m2':'preuM2'
    
    
},inplace=True

)

C:\Users\chuec\AppData\Local\Temp\ipykernel_7432\651114907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precios_2014.rename(columns = {


In [59]:
df_precios_2014.head(2)

,nomBarri,Anio,preuMitja,preuM2
219,el raval,2014-01-01,1189,2.176
220,el barri gòtic,2014-01-01,1325,2.632


In [60]:
df_precios_2014.isnull().sum()

nomBarri      0
Anio          0
preuMitja     0
preuM2       15
dtype: int64

In [61]:
df_precios_2014.describe(exclude = np.number)

,nomBarri,Anio,preuMitja
count,73,73,73
unique,73,NaN,54
top,el raval,NaN,--
freq,1,NaN,16
mean,NaN,2014-01-01 00:00:00,NaN
min,NaN,2014-01-01 00:00:00,NaN
25%,NaN,2014-01-01 00:00:00,NaN
50%,NaN,2014-01-01 00:00:00,NaN
75%,NaN,2014-01-01 00:00:00,NaN
max,NaN,2014-01-01 00:00:00,NaN


In [62]:
df_precios_2014[df_precios_2014['preuM2'].isnull()]

,nomBarri,Anio,preuMitja,preuM2
230,la marina del prat vermell - zona franca,2014-01-01,--,NaN
239,pedralbes,2014-01-01,--,NaN
240,vallvidrera el tibidabo i les planes,2014-01-01,--,NaN
254,la font d'en fargues,2014-01-01,--,NaN
257,sant genís dels agudells,2014-01-01,--,NaN
258,montbau,2014-01-01,--,NaN
259,la vall d'hebron,2014-01-01,--,NaN
260,la clota,2014-01-01,--,NaN
265,can peguera,2014-01-01,--,NaN
266,la guineueta,2014-01-01,766,NaN


In [63]:
# Para convertir los valores -- en nulos y poder imputarlos
df_precios_2014['preuMitja'] = df_precios_2014['preuMitja'].apply(pd.to_numeric, errors='coerce')


C:\Users\chuec\AppData\Local\Temp\ipykernel_7432\485812093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precios_2014['preuMitja'] = df_precios_2014['preuMitja'].apply(pd.to_numeric, errors='coerce')


In [64]:
df_precios_2014.isnull().sum()

nomBarri      0
Anio          0
preuMitja    16
preuM2       15
dtype: int64

In [65]:
df_precios_2014[df_precios_2014['nomBarri'] == 'sant pere santa caterina i la ribera']

,nomBarri,Anio,preuMitja,preuM2
222,sant pere santa caterina i la ribera,2014-01-01,1225.0,2.482


In [66]:
df_cens_pivot['nomBarri'] = df_cens_pivot['nomBarri'].str.lower()

In [67]:
# merge del dataframe original con el dataframe de price. Comprobamos que no se haga un cartesiano
print(df_cens_pivot.shape)

df_cens_pivot = pd.merge(df_cens_pivot,df_precios_2014, on = ['nomBarri','Anio'], how = 'left')

print(df_cens_pivot.shape)

(73, 14)
(73, 16)


In [68]:
df_cens_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   codiDistricte          73 non-null     object        
 1   nomDistricte           73 non-null     object        
 2   codiBarri              73 non-null     object        
 3   nomBarri               73 non-null     object        
 4   nAltres                73 non-null     int64         
 5   nAutomocio             73 non-null     int64         
 6   nEquipamentPersonal    73 non-null     int64         
 7   nLocalBuit             73 non-null     int64         
 8   nOciCultura            73 non-null     int64         
 9   nParamentLlar          73 non-null     int64         
 10  nQuotidiaAlimentari    73 non-null     int64         
 11  nQuotidiaNoAlimentari  73 non-null     int64         
 12  nServeis               73 non-null     int64         
 13  Anio   

In [69]:
df_cens_pivot.head()

,codiDistricte,nomDistricte,codiBarri,nomBarri,nAltres,nAutomocio,nEquipamentPersonal,nLocalBuit,nOciCultura,nParamentLlar,nQuotidiaAlimentari,nQuotidiaNoAlimentari,nServeis,Anio,preuMitja,preuM2
0,01,Ciutat Vella,01,el raval,381,1,225,245,88,137,296,47,1120,2014-01-01,1189.0,2.176
1,01,Ciutat Vella,02,el barri gòtic,315,0,478,199,78,76,161,37,786,2014-01-01,1325.0,2.632
2,01,Ciutat Vella,03,la barceloneta,99,0,26,108,24,13,89,16,370,2014-01-01,1648.0,2.135
3,01,Ciutat Vella,04,"sant pere, santa caterina i la ribera",259,1,363,155,53,98,175,34,688,2014-01-01,NaN,NaN
4,02,Eixample,05,el fort pienc,81,12,40,199,47,55,113,33,556,2014-01-01,921.0,2.224


In [70]:
df_cens_pivot.isnull().sum()

codiDistricte             0
nomDistricte              0
codiBarri                 0
nomBarri                  0
nAltres                   0
nAutomocio                0
nEquipamentPersonal       0
nLocalBuit                0
nOciCultura               0
nParamentLlar             0
nQuotidiaAlimentari       0
nQuotidiaNoAlimentari     0
nServeis                  0
Anio                      0
preuMitja                23
preuM2                   22
dtype: int64

In [71]:
# Lista de columnas numéricas para las cuales quieres imputar los nulos
columnas_numericas = ['nAltres', 'nAutomocio', 'nEquipamentPersonal', 'nLocalBuit', 'nOciCultura', 'nParamentLlar', 'nQuotidiaAlimentari', 'nQuotidiaNoAlimentari', 'nServeis', 'preuMitja', 'preuM2']

# Calcula la media por cada grupo de 'nomDistricte' solo para las columnas numéricas
means_by_districte = df_cens_pivot.groupby('nomDistricte')[columnas_numericas].mean()

# Itera sobre las filas de tu DataFrame y completa los valores nulos
for index, row in df_cens_pivot.iterrows():
    for column in columnas_numericas:
        if pd.isnull(row[column]):
            # Utiliza la media del grupo correspondiente para imputar el valor nulo
            df_cens_pivot.at[index, column] = means_by_districte.at[row['nomDistricte'], column]

# Ahora, todos los valores nulos deberían estar imputados con la media del grupo correspondiente

In [72]:
df_cens_pivot.isnull().sum()

codiDistricte            0
nomDistricte             0
codiBarri                0
nomBarri                 0
nAltres                  0
nAutomocio               0
nEquipamentPersonal      0
nLocalBuit               0
nOciCultura              0
nParamentLlar            0
nQuotidiaAlimentari      0
nQuotidiaNoAlimentari    0
nServeis                 0
Anio                     0
preuMitja                0
preuM2                   0
dtype: int64

In [77]:
# Vemos que los precios por m2 2€ el m2 no hace mucho sentido, pero si al multiplicarlo por 10, 20€ el m2 para un local comercial en BCN, cobra mayor sentido
df_cens_pivot['preuM2'] = df_cens_pivot['preuM2']*10

In [79]:
df_cens_pivot.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
nAltres,73.0,85.986301,87.741016,2.0,23.000000,57.00,106.000000,387.00
nAutomocio,73.0,4.931507,7.459760,0.0,0.000000,2.00,6.000000,33.00
nEquipamentPersonal,73.0,70.054795,120.589396,0.0,5.000000,26.00,77.000000,622.00
nLocalBuit,73.0,133.082192,124.538231,0.0,38.000000,108.00,172.000000,539.00
nOciCultura,73.0,38.328767,47.287259,0.0,6.000000,23.00,47.000000,228.00
nParamentLlar,73.0,42.273973,47.980106,0.0,6.000000,28.00,55.000000,213.00
nQuotidiaAlimentari,73.0,89.986301,84.393834,0.0,25.000000,61.00,116.000000,390.00
nQuotidiaNoAlimentari,73.0,23.616438,21.011657,0.0,8.000000,18.00,32.000000,92.00
nServeis,73.0,421.643836,414.566252,2.0,120.000000,307.00,584.000000,1861.00
preuMitja,73.0,909.256980,187.891703,573.0,795.000000,837.00,947.333333,1648.00


In [78]:
df_cens_pivot.describe(exclude=np.number).T

,count,unique,top,freq,mean,min,25%,50%,75%,max
codiDistricte,73,10,08,13,NaT,NaT,NaT,NaT,NaT,NaT
nomDistricte,73,10,Nou Barris,13,NaT,NaT,NaT,NaT,NaT,NaT
codiBarri,73,73,01,1,NaT,NaT,NaT,NaT,NaT,NaT
nomBarri,73,73,el raval,1,NaT,NaT,NaT,NaT,NaT,NaT
Anio,73,NaN,NaN,NaN,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01,2014-01-01


# TO JSON

In [80]:
# convertimos nuestro dataset a formato json para pasar al equipo de Back
df_censo_json_v3 = df_cens_pivot.to_json(orient='records', lines=True)

In [82]:
file_path = "..\outputs\df_censo_json_v3.json"

with open(file_path, 'w') as file:
    file.write(df_censo_json_v3)